In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
import time
import os
import datetime

chrome_path = "selenium_driver_chrome/chromedriver.exe" 
gmailsername = 'Enter yours'
gmailpassword = 'Enter yours'

web = webdriver.Chrome(chrome_path)

StockIDList = ['0056','0050','2881','2886','2892','2884','2891',
               '2887','5820','5530','2412','2882','2880','5880',
               '8926','6183','1722','1737']

ChartList = {}

pd.options.display.max_colwidth = 100
df = pd.DataFrame(columns=['ChineseName','OpenValue','CloseValue','Kvalue','Dvalue','RSI5value','RSI10value', 'URL'])

#web.set_window_position(0,0) #瀏覽器位置
#web.set_window_size(700,700) #瀏覽器大小


In [2]:
def GetPrintScreen(StockID):
    web.execute_script("document.body.style.zoom='60%'")
    web.execute_script("window.scrollTo(0, 370)") 
    FileName = 'Candlestick Chart/' + StockID + "_" + datetime.date.today().strftime("%Y%m%d") + '.png'
    web.save_screenshot(FileName) 
    ChartList[StockID] = FileName
    
def ParseDataNewRow(df, StockID):
    URL = 'https://www.wantgoo.com/stock/astock/techchart?StockNo=' + StockID
    web.get(URL)
    time.sleep(1)
    GetPrintScreen(StockID)
    ChineseName = web.find_element_by_xpath('//*[@id="container"]/ol/li[3]/a/span').text
    OpenValue = web.find_element_by_xpath('//*[@id="container"]/div[3]/div[2]/ul/li[1]/b').text
    CloseValue = web.find_element_by_xpath('//*[@id="container"]/div[3]/div[2]/ul/li[4]/b').text
    Kvalue = web.find_element_by_xpath('//*[@id="V_K9"]').text
    Dvalue = web.find_element_by_xpath('//*[@id="V_D9"]').text
    RSI5value = web.find_element_by_xpath('//*[@id="V_RSI5"]').text
    RSI10value = web.find_element_by_xpath('//*[@id="V_RSI10"]').text
    dftemp = pd.DataFrame([[ChineseName, OpenValue, CloseValue, Kvalue, Dvalue, RSI5value, RSI10value, URL]], columns = df.columns)
    return dftemp


for StockID in StockIDList:
    dftemp = ParseDataNewRow(df, StockID)
    df = df.append(dftemp, ignore_index=True)
    print("Finish: " + StockID)

df = df.sort_values(by=['Kvalue'])

Finish: 0056
Finish: 0050
Finish: 2881
Finish: 2886
Finish: 2892
Finish: 2884
Finish: 2891
Finish: 2887
Finish: 5820
Finish: 5530
Finish: 2412
Finish: 2882
Finish: 2880
Finish: 5880
Finish: 8926
Finish: 6183
Finish: 1722
Finish: 1737


In [3]:
filename = "KD-overview/KD{0}.csv"
oldfilename = filename.format('')
while (os.path.isfile(oldfilename)):
    v= datetime.datetime.fromtimestamp(os.path.getmtime(oldfilename))
    x = v.strftime('%Y%m%d-%H%M%S')
    try:
        os.rename(oldfilename,filename.format(x))
    except FileExistsError:
        os.remove(filename.format(x))
        os.rename(oldfilename,filename.format(x))

df.to_csv(oldfilename, encoding="big5")

In [ ]:
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
from email.mime.multipart import MIMEMultipart
from email.mime.image import MIMEImage
from smtplib import SMTP
import smtplib
import sys



recipients = ['a4455669@gmail.com']#, 'iverson0912@livemail.tw', 'joannechiuu@gmail.com'] 
emaillist = [elem.strip().split(',') for elem in recipients]
msg = MIMEMultipart()
msg['Subject'] = "KD Daily Summary"
msg['From'] = 'a4455669@gmail.com'


html = """\
<html>
  <head> </head>
  <body>
    {0}
  </body>
</html>
""".format(df.to_html())

part1 = MIMEText(html, 'html')
msg.attach(part1)

for StockID in ChartList.keys():
    img_data = open(ChartList[StockID], 'rb').read()
    image = MIMEImage(img_data, name=StockID)
    print('Embedding: ' + StockID)
    msg.attach(image)

server = smtplib.SMTP('smtp.gmail.com:587')
server.ehlo()
server.starttls()
server.login(gmailsername,gmailpassword)
server.sendmail(msg['From'], emaillist , msg.as_string())

Embedding: 0056
Embedding: 0050
Embedding: 2881
Embedding: 2886
Embedding: 2892
Embedding: 2884
Embedding: 2891
Embedding: 2887
Embedding: 5820
Embedding: 5530
Embedding: 2412
Embedding: 2882
Embedding: 2880
Embedding: 5880
Embedding: 8926
Embedding: 6183
Embedding: 1722
Embedding: 1737


In [ ]:
web.quit()